In [10]:
#built in 
import os 
import time 
import shutil 
import yaml 
#remote 
import torch

from ultralytics import YOLO


#local 
from dataset.GenerateSyntheticDataset import generate_dataset

In [11]:
#default training checks 
TRAIN = True
GENERATE = True 


#Parameters for  generating  dataset 
NUMBER_OF_IMAGES =10
#segmentation  or detection mode 
SEG =False 
#dataset dir 
SYNTHETIC_DATASET_DIR ='dataset/GeneratedDataset'




# Parameters for training 
EPOCHS  = 2
YAML_DATA_PATH = 'data.yaml'

In [41]:


def add_class(path_yaml_file:str,new_value,key)->None:
    """
    ADDS CLASS  TO YAML FILE 
    """
    with open(path_yaml_file) as f:
            dic =yaml.load(f,yaml.FullLoader)
           
            if(key in list(dic['names'].keys())):
                print(f"[ERROR] Cant use this key  is already being used {key} ")
            else:
                dic['names'][key]=new_value
            f.close()
            file = open(path_yaml_file,'w')
            yaml.dump(dic,file)
            file.close()
           



In [42]:
add_class(YAML_DATA_PATH,'club_mate',69)





[ERROR] Cant use this key  is already being used 69 


In [ ]:
if (GENERATE == True ):
    if(os.path.isdir(SYNTHETIC_DATASET_DIR)):
        shutil.rmtree(SYNTHETIC_DATASET_DIR)
    generate_dataset(NUMBER_OF_IMAGES,SEG)
    

In [ ]:
def train(model_path,seg =True)->None:
    
    
    if(seg==False):
              # use 'yolov8n.pt' to start fresh
            try:
                model = YOLO(model_path)
            except FileNotFoundError:
                print("Model not found, retraining from scratch with YoloV8n")
                model = YOLO('yolov8n.pt',task='detect')
    if(seg == True):
            
            try:
                model = YOLO(model_path)
            except FileNotFoundError:
                print("Model not found, retraining from scratch with YoloV8n")
                model = YOLO('yolov8n-seg.pt',task= 'segment')

     #check if using cuda
    if torch.cuda.is_available():
        print("Using GPU")
        model.cuda()
            
        
    start = time.time()
       
        # if(seg == True ):
        #     results = model.train(data=YAML_DATA_PATH , epochs=EPOCHS)
        # else:
        #      results = model.train(data=YAML_DATA_PATH , epochs=EPOCHS)
    results =model.train(data=YAML_DATA_PATH , epochs=EPOCHS)
       
    end = time.time()
        
       
    print("Training finished")
    val_metrics = model.val()
    val_metrics

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/

In [ ]:
if(TRAIN ==True): 
    #can be any path
    train('runs/detect/train/weights/best.pt',seg=SEG)